# 要安装的东西

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
%cd  "/content/drive/MyDrive/memoire"

/content/drive/MyDrive/memoire


In [4]:
! pip install datasets
! pip install transformers
! pip install evaluate
! pip install accelerate
! pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.7 MB/s 
     |████████████████████████████████| 212 kB 47.4 MB/s 
     |████████████████████████████████| 120 kB 52.3 MB/s 
     |████████████████████████████████| 115 kB 62.6 MB/s 
     |████████████████████████████████| 127 kB 46.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.0 MB/s 
     |████████████████████████████████| 6.6 MB 32.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [5]:
from huggingface_hub import notebook_login

notebook_login()

# 数据准备

In [ ]:
from datasets import load_dataset
import transformers

In [ ]:
dataset = load_dataset("csv", data_files=["./clean_data/MNLI.csv", "./clean_data/SNLI.csv", "./clean_data/RTE.csv"], sep='\t', split="train")
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
dataset = dataset.rename_column("Unnamed: 0", "id")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'positif', 'negatif'],
        num_rows: 2700
    })
    test: Dataset({
        features: ['id', 'positif', 'negatif'],
        num_rows: 300
    })
})

# 模型和工具准备

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = 'minwhoo/bart-base-negative-claim-generation'
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to('cuda' if torch.cuda.is_available() else 'cpu')

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [ ]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [ ]:
prefix = "negative sentences generation: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["positif"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["negatif"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
from transformers import DataCollatorForSeq2Seq

tokenized_datasets = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
import evaluate

def compute_metrics(eval_preds):
    metric = evaluate.load("sacrebleu") # 这里可以换成其他的吗？
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
from transformers import Seq2SeqTrainingArguments

# # 第一种
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     per_device_train_batch_size = 2,
#     per_device_eval_batch_size = 2,
#     num_train_epochs=3,
# )

# 第二种
training_args = Seq2SeqTrainingArguments(
    f"bart-base-finetuned-negative-sentences-generation",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Seq2SeqTrainer

# # without recall
# trainer = Seq2SeqTrainer(
#     model,
#     training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
# )

# with recall
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/content/drive/MyDrive/memoire/bart-base-finetuned-negative-sentences-generation is already a clone of https://huggingface.co/oceaneqiu/bart-base-finetuned-negative-sentences-generation. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: positif, id, negatif. If positif, id, negatif are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2700
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 4050


Epoch,Training Loss,Validation Loss,Bleu
1,0.117900,0.086858,75.566118
2,0.062500,0.083108,75.785863
3,0.037300,0.085889,76.281923


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: positif, id, negatif. If positif, id, negatif are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 2
Saving model checkpoint to bart-base-finetuned-negative-sentences-generation/checkpoint-1350
Configuration saved in bart-base-finetuned-negative-sentences-generation/checkpoint-1350/config.json
Model weights saved in bart-base-finetuned-negative-sentences-generation/checkpoint-1350/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-negative-sentences-generation/checkpoint-1350/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-negative-sentences-generation/checkpoint-1350/special_tokens_map.json
tokenizer config file saved in bart-base-finetuned-negative-sentences-generation/tokenizer_config.json
S

TrainOutput(global_step=4050, training_loss=0.08402228028685958, metrics={'train_runtime': 470.2804, 'train_samples_per_second': 17.224, 'train_steps_per_second': 8.612, 'total_flos': 119669116170240.0, 'train_loss': 0.08402228028685958, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub(f"bart-base-finetuned-negative-sentences-generation")

Saving model checkpoint to bart-base-finetuned-negative-sentences-generation
Configuration saved in bart-base-finetuned-negative-sentences-generation/config.json
Model weights saved in bart-base-finetuned-negative-sentences-generation/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-negative-sentences-generation/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-negative-sentences-generation/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Bleu', 'type': 'bleu', 'value': 76.28192290000447}]}


# 用简单的例子来验证一下是不是自己想要的模型

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
checkpoint = './bart-base-finetuned-negative-sentences-generation/checkpoint-4050'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.to('cuda' if torch.cuda.is_available() else 'cpu')

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [ ]:
examples = [
    "Unless the report is restricted by law or regulation, auditors should ensure that copies be made available for public inspection.",
    "um-hum they have socialized socialized health care",
    "IDAs are special in that low-income savers receive matching funds from federal and state governments as well as private sector organizations as an incentive to save.",
    "I sometimes had the desire to be a chef",
    "Well, I've never had a secret hankering to be a detective!"
]

batch = tokenizer(examples, max_length=128, padding=True, truncation=True, return_tensors="pt")
out = model.generate(batch['input_ids'].to(model.device), max_length=128)
negative_examples = tokenizer.batch_decode(out, skip_special_tokens=True)
negative_examples

['Unless the report is restricted by law or regulation, auditors should not ensure that copies be made available for public inspection.',
 'um-hum they have not socialized socialized health care',
 'IDAs are not special in that low-income savers receive matching funds from federal and state governments as well as private sector organizations as an incentive to save.',
 'I sometimes did not have the desire to be a chef',
 "Well, I've never had a secret hankering to be a detective!"]

In [ ]:
examples = [
    "Unless the report is restricted by law or regulation, auditors should ensure that copies be made available for public inspection.",
    "um-hum they have socialized socialized health care",
    "IDAs are special in that low-income savers receive matching funds from federal and state governments as well as private sector organizations as an incentive to save.",
    "I sometimes had the desire to be a chef",
    "Well, I've never had a secret hankering to be a detective!"
]

batch = tokenizer(examples, max_length=128, padding=True, truncation=True, return_tensors="pt")

print(batch)

In [7]:
example = "Unless the report is restricted by law or regulation, auditors should ensure that copies be made available for public inspection"
batch = tokenizer(example, max_length=128, padding=True, truncation=True, return_tensors="pt")
batch

{'input_ids': tensor([[    0, 33757,     5,   266,    16,  9393,    30,   488,    50,  5746,
             6,  9818,  9314,   197,  1306,    14, 11288,    28,   156,   577,
            13,   285,  9472,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [9]:
tokens = tokenizer.convert_ids_to_tokens(batch["input_ids"].tolist()[0])
print(tokens)
tokens

['<s>', 'Unless', 'Ġthe', 'Ġreport', 'Ġis', 'Ġrestricted', 'Ġby', 'Ġlaw', 'Ġor', 'Ġregulation', ',', 'Ġaud', 'itors', 'Ġshould', 'Ġensure', 'Ġthat', 'Ġcopies', 'Ġbe', 'Ġmade', 'Ġavailable', 'Ġfor', 'Ġpublic', 'Ġinspection', '</s>']


['<s>',
 'Unless',
 'Ġthe',
 'Ġreport',
 'Ġis',
 'Ġrestricted',
 'Ġby',
 'Ġlaw',
 'Ġor',
 'Ġregulation',
 ',',
 'Ġaud',
 'itors',
 'Ġshould',
 'Ġensure',
 'Ġthat',
 'Ġcopies',
 'Ġbe',
 'Ġmade',
 'Ġavailable',
 'Ġfor',
 'Ġpublic',
 'Ġinspection',
 '</s>']

In [10]:
out = model.generate(batch['input_ids'].to(model.device), max_length=128)
out

tensor([[    2,     0, 33757,     5,   266,    16,  9393,    30,   488,    50,
          5746,     6,  9818,  9314,   197,    45,  1306,    14, 11288,    28,
           156,   577,    13,   285,  9472,     2]], device='cuda:0')

In [11]:
negative_example = tokenizer.batch_decode(out, skip_special_tokens=True)
negative_example

['Unless the report is restricted by law or regulation, auditors should not ensure that copies be made available for public inspection']

In [ ]:
out = model.generate(batch['input_ids'].to(model.device), max_length=128)

In [ ]:
print(out)

tensor([[    2,     0, 33757,     5,   266,    16,  9393,    30,   488,    50,
          5746,     6,  9818,  9314,   197,    45,  1306,    14, 11288,    28,
           156,   577,    13,   285,  9472,     4,     2,     1,     1,     1,
             1,     1,     1,     1,     1],
        [    2,     0,   783,    12, 18257,    51,    33,    45,   592,  1538,
           592,  1538,   474,   575,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1],
        [    2,     0,  2688,  1620,    32,    45,   780,    11,    14,   614,
            12,  7214,  2241,  3697,  1325,  8150,  1188,    31,   752,     8,
           194,  3233,    25,   157,    25,   940,  1293,  2665,    25,    41,
         10814,     7,  1871,     4,     2],
        [    2,     0,   100,   393,    56,     5,  4724,     7,    28,    10,
          8172,     2,     1,     1,     1,     1,     1,     1,     1,    

In [ ]:
out.shape

torch.Size([5, 35])

In [ ]:
negative_examples = tokenizer.batch_decode(out, skip_special_tokens=True)
negative_examples

['Unless the report is restricted by law or regulation, auditors should not ensure that copies be made available for public inspection.']

# 验证test

## 在一个数据上试验

In [ ]:
import string

test = tokenized_datasets["test"]
device = 'cuda'
preds = model.generate(input_ids = torch.tensor(test[1]['input_ids']).to(device).view(1,-1), max_length=128)
prd = tokenizer.decode(np.array(preds.cpu()[0]))[7:-4]
reals = tokenizer.decode(np.array(test[1]['labels']))[3:-4]
print(prd)
print(reals)
print(prd==reals)


Every tube did not contain a body, and every body was a famous figure from history.
Every tube did not contain a body, and every body was a famous figure from history.
True


## 测试集上面测试

In [ ]:
import numpy as np
import string

test = tokenized_datasets["test"]
count = 0
for index in range(len(test)):
  device = 'cuda'
  preds = model.generate(input_ids = torch.tensor(test[index]['input_ids']).to(device).view(1,-1), max_length=128)
  neg_pred = tokenizer.decode(np.array(preds.cpu()[0]))[7:-4].strip(string.punctuation)
  neg_refr = tokenizer.decode(np.array(test[index]['labels']))[3:-4].strip(string.punctuation)
  count+=1.0*(neg_pred.strip() == neg_refr.strip())
  if index%10==0:
    print(neg_pred.strip())
    print(neg_refr.strip())
count/len(test)

# results/500上的准确度是：0.7933333333333333
# results/4000上的准确度是：0.9466666666666667
# results2/2025的准确度是：0.8233333333333334


Self-sufficiency has not been turned into a formal public awareness campaign in San Francisco, by Mayor Gavin Newsom
Self-sufficiency has not been turned into a formal public awareness campaign in San Francisco, by Mayor Gavin Newsom
Prior to that, Andy Sipowicz did not begin to date Assistant District Attorney Sylvia Costas. They got married in 1995 and had a son Theo in 1996
Prior to that, Andy Sipowicz began to date Assistant District Attorney Sylvia Costas. They did not get married in 1995 and had a son Theo in 1996
A woman is not standing up
A woman is not standing up
The woman is not selling wheat
The woman is not selling wheat
Postal Service's domestic rates for First-Class and Priority Mail to the inbound mail distribution did not describe above
Postal Service's domestic rates for First-Class and Priority Mail to the inbound mail distribution are not described above
Democrats did not win the elections three years ago
Democrats did not win the elections three years ago
The man n

0.8233333333333334